# Общая информация
__Цель:__ сделать fine-tuning GPT neo

# Начальная инициализация

## Установка библиотек

In [3]:
#!pip install bert_score datasets rouge_score evaluate pynvml transformers wandb

## Подключение гугл диска

In [1]:

from utils import *

In [20]:
if 'google.colab' in str(get_ipython()):
    !pip install bert_score datasets rouge_score evaluate pynvml transformers wandb
    
    !wget https://raw.githubusercontent.com/Myashka/Diploma/master/Notebooks/utils.py

    from google.colab import drive
    drive.mount("/content/drive")
else:
    import sys
    sys.path.append(r"D:\vkr\Notebooks")

from utils import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-12-11 21:52:20--  https://raw.githubusercontent.com/Myashka/Diploma/master/Notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8246 (8.1K) [text/plain]
Saving to: ‘utils.py.1’

utils.py.1          100%[===================>]   8.05K  --.-KB/s    in 0s      

2022-12-11 21:52:20 (96.2 MB/s) - ‘utils.py.1’ saved [8246/8246]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pickle
import warnings

import nltk
import numpy as np
import pandas as pd
import torch
from datasets import load_metric
from evaluate import load
from torch.utils.data import Dataset, random_split
from tqdm.auto import tqdm
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          EarlyStoppingCallback, Trainer, TrainingArguments)

warnings.filterwarnings("ignore")
from nltk.translate.bleu_score import sentence_bleu

In [3]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [4]:
print_gpu_utilization()

GPU memory occupied: 0 MB.


## Фиксация сидов

In [5]:
torch.manual_seed(42)
np.random.seed(42)

# Данные

## Чтение данных

In [6]:
data_path = r'/content/drive/MyDrive/Colab Notebooks/vkr_data/data/python_clean_200_df.pkl'

In [7]:
#%pip install pandas==1.4.1

In [8]:
df = pd.read_pickle(data_path)

In [9]:
pd.set_option("display.max_colwidth", None)

In [13]:
len(df)

1553

In [12]:
# pd.set_option("display.max_colwidth", None)
# with open( f'../../data/filtered_df.p', 'rb') as f:
#     df = pickle.load(f)

## Разбиение на train test

In [14]:
df = df.sort_values("Q_date_open")
train_df, test_df = np.split(df, [int(0.85 * len(df))])

# Загрзка модели и токенизатора

In [15]:
device = torch.device("cuda") if torch.cuda.is_available else "cpu"

In [16]:
model_name = "EleutherAI/gpt-neo-1.3B"

In [17]:
model_name = "EleutherAI/gpt-neo-1.3B"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [18]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 2048)

## Создание датасета

In [19]:
#promt_1 = """You will be asked a question. For this question, you must answer by its title and question body.\nTitle: """
promt_1 = ''
promt_2 = '\nQuestion: '
#promt_2 = ''
promt_3 = '\nAnswer:'
promts = [promt_1, promt_2, promt_3]

use_title = False
use_question = True

In [20]:
train_dataset = Q_A_Dataset(train_df, tokenizer, *promts, use_title=use_title, use_question=use_question)
test_dataset = Q_A_Dataset(test_df, tokenizer, *promts, use_title=use_title, use_question=use_question)

In [21]:
del df, train_df, test_df

# Инициализация в WandB

In [22]:
import wandb

wandb.login()
run = wandb.init(project="QA specific domain", entity="myashka")

%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
%env WANDB_SILENT=true

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: myashka. Use `wandb login --relogin` to force relogin


env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all
env: WANDB_SILENT=true


### Заморозка весов

In [23]:
for n, p in model.named_parameters():
    if "transformer.h" in n:
        layer_num = int(n.split(".")[2])
        if "ln_" not in n and layer_num > 0 and layer_num < 23:
            p.requires_grad = False
            print("Freeze", n)

Freeze transformer.h.1.attn.attention.k_proj.weight
Freeze transformer.h.1.attn.attention.v_proj.weight
Freeze transformer.h.1.attn.attention.q_proj.weight
Freeze transformer.h.1.attn.attention.out_proj.weight
Freeze transformer.h.1.attn.attention.out_proj.bias
Freeze transformer.h.1.mlp.c_fc.weight
Freeze transformer.h.1.mlp.c_fc.bias
Freeze transformer.h.1.mlp.c_proj.weight
Freeze transformer.h.1.mlp.c_proj.bias
Freeze transformer.h.2.attn.attention.k_proj.weight
Freeze transformer.h.2.attn.attention.v_proj.weight
Freeze transformer.h.2.attn.attention.q_proj.weight
Freeze transformer.h.2.attn.attention.out_proj.weight
Freeze transformer.h.2.attn.attention.out_proj.bias
Freeze transformer.h.2.mlp.c_fc.weight
Freeze transformer.h.2.mlp.c_fc.bias
Freeze transformer.h.2.mlp.c_proj.weight
Freeze transformer.h.2.mlp.c_proj.bias
Freeze transformer.h.3.attn.attention.k_proj.weight
Freeze transformer.h.3.attn.attention.v_proj.weight
Freeze transformer.h.3.attn.attention.q_proj.weight
Freeze t

In [24]:
print_gpu_utilization()

GPU memory occupied: 5910 MB.


# Trainer и обучение

In [25]:
model.config.use_cache = False

In [27]:
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/vkr_data/checkpoints",
    num_train_epochs=5,
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy="steps",
    save_steps=45,
    evaluation_strategy="steps",
    eval_steps=15,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    report_to="wandb",  # enable logging to W&B
    run_name="gpt_neo_android_api_change",
    gradient_checkpointing=True,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=12)],
    data_collator=lambda data: {
        "input_ids": collate_batch([f[0] for f in data], tokenizer),
        "attention_mask": collate_batch(
            [f[1] for f in data], tokenizer, "attention_mask"
        ),
        "labels": collate_batch([f[2] for f in data], tokenizer),
    },
)

PyTorch: setting up devices
Using cuda_amp half precision backend


In [28]:
trainer.train()

***** Running training *****
  Num examples = 1320
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1650
  Number of trainable parameters = 208009216
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
15,4.270000,3.627420
30,1.956800,1.565861
45,1.466700,1.467479
60,1.326700,1.428972
75,1.260900,1.426681
90,1.268800,1.430890
105,1.368400,1.436673
120,1.371800,1.441947
135,1.631800,1.443401
150,1.155200,1.443104


***** Running Evaluation *****
  Num examples = 233
  Batch size = 4
***** Running Evaluation *****
  Num examples = 233
  Batch size = 4
***** Running Evaluation *****
  Num examples = 233
  Batch size = 4
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/vkr_data/checkpoints/checkpoint-45
Configuration saved in /content/drive/MyDrive/Colab Notebooks/vkr_data/checkpoints/checkpoint-45/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/vkr_data/checkpoints/checkpoint-45/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 233
  Batch size = 4
***** Running Evaluation *****
  Num examples = 233
  Batch size = 4
***** Running Evaluation *****
  Num examples = 233
  Batch size = 4
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/vkr_data/checkpoints/checkpoint-90
Configuration saved in /content/drive/MyDrive/Colab Notebooks/vkr_data/checkpoints/checkpoint-90/config.json
Model weights saved in /content/drive/MyDrive/Colab

TrainOutput(global_step=270, training_loss=1.5959062258402505, metrics={'train_runtime': 1337.8378, 'train_samples_per_second': 4.933, 'train_steps_per_second': 1.233, 'total_flos': 2633668522573824.0, 'train_loss': 1.5959062258402505, 'epoch': 0.82})

In [25]:
wandb.finish()

eval/loss,█▂▁▁▁▁▁▁▁
eval/runtime,▁▆█▇▁▁█▇▇
eval/samples_per_second,█▂▁▂██▁▂▂
eval/steps_per_second,█▂▁▂██▁▂▂
train/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▇▇▇███
train/learning_rate,▁▂▂▃▄▅▅▆▇████
train/loss,█▇▄▂▂▁▁▂▁▁▁▂▂
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


# Evaluation

## Подгрузка нужной модели (опичионально)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Colab Notebooks/vkr_data/results/checkpoint-90"
).to(device)

In [29]:
model.config.use_cache = True

### Настройка wandb

In [30]:
run_table_name = 'base_tuning_python_comparision'
run_table = wandb.Artifact(f"{run_table_name}", type="run_table")

In [31]:
evaluator = Evaluator(run_table, model, tokenizer)

In [32]:
eval_table, bleu, rouge, bert_pr, bert_rec, bert_f1 = evaluator.evaluate(test_dataset, *promts, use_title=use_title, use_question=use_question)

  0%|          | 0/233 [00:00<?, ?it/s]

Could not locate the tokenizer configuration file, will try to use the model config instead.


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/pytorch_model.bin
All the weights of RobertaModel were initialized from the model checkpoint at roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaModel for predictions without further training.


In [33]:
eval_filename = 'tuned_model_evaluation_python_200'

In [34]:
run_table.add(eval_table, f"{eval_filename}")
run.log_artifact(run_table)
wandb.finish()

eval/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▆▆▁▁▆▇▁▁█▇▁▁▆▆▂▇▆
eval/samples_per_second,█▂▂▇▇▂▁▇▇▁▂▇█▂▂▇▂▂
eval/steps_per_second,█▂▂▇▇▂▁▇▇▁▂▇█▂▂▇▂▂
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/learning_rate,▁▂▂▃▄▅▅▆▇████████████▇▇▇▇▇▇
train/loss,█▇▃▂▂▁▁▂▁▁▂▁▂▁▁▂▂▁▁▂▂▂▂▁▁▂▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
